In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #it is a submodule in tensorflow keras that provides helpertools specifically for working with images. imagedatagenerator is the main tool for real time data augumentation and rescalling images.
from tensorflow.keras.models import Sequential#models is a submodule that contains different ways to build a model . sequential is a specific type of model provided by keras.it allows us to stag layers 1 after another from input to output in a straight line.sequential means the model follows step by step like a pipeline.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout#layers is a submodule which is used in building a CNN model.Conv2D is used for extracting features from images and it works by sliding filters by looking at small parts of an image to detect patterns. Dense creates a fully connected neural network by getting the input from flatten it is the brain of the network and each neuron is connect to all the neurons in the previous layer.
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
with ZipFile('/content/drive/MyDrive/wheat_disease.zip','r')as zipObj:
  zipObj.extractall('/content/drive/MyDrive')
  print('Dataset has been extracted')

Dataset has been extracted


In [ ]:
# Set image size and paths
IMG_SIZE = 256   #resizing the image to 256by256
BATCH_SIZE = 32
DATASET_PATH = '/content/drive/MyDrive/wheat_leaf'

In [ ]:
#simple image loader without augmentation
datagen = ImageDataGenerator(
    rescale=1./255, # we want our image value between 0 and 1
    validation_split = 0.2 #20% validation split
    )


In [ ]:
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)




Found 327 images belonging to 3 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 80 images belonging to 3 classes.


In [ ]:
#CNN model
model = Sequential([
Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
MaxPooling2D((2, 2)),
Conv2D(64, (3, 3), activation='relu'),
MaxPooling2D((2, 2)),
Conv2D(128, (3, 3), activation='relu'),
MaxPooling2D((2, 2)),
Flatten(),
Dense(128, activation='relu'),
Dropout(0.5),
Dense(3, activation='softmax')]) #3classes: lung_n, lung_aca, lung_scc

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Show model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Early stopping
early_stop=EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
#train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 77s 6s/step - accuracy: 0.4561 - loss: 2.0526 - val_accuracy: 0.5250 - val_loss: 1.0933
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 56s 5s/step - accuracy: 0.5834 - loss: 0.8895 - val_accuracy: 0.5125 - val_loss: 0.9062
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 61s 6s/step - accuracy: 0.6512 - loss: 0.7486 - val_accuracy: 0.5750 - val_loss: 0.8535
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.6556 - loss: 0.7539 - val_accuracy: 0.6625 - val_loss: 0.9219
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - accuracy: 0.7288 - loss: 0.5541 - val_accuracy: 0.7625 - val_loss: 0.6446
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.7782 - loss: 0.4769 - val_accuracy: 0.8375 - val_loss: 0.3939
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.8013 - loss: 0.3622 - val_accuracy: 0.6500 - val_loss: 0.8325
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.8277 - loss: 0.3780 - val_accuracy: 0.6000 - val_loss:

In [ ]:
#Save the model
model.save('wheat_disease.h5')

In [ ]:
from google.colab import files
files.download('wheat_disease.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import cv2
import tensorflow as tf


In [ ]:
#load model
model = tf.keras.models.load_model('/content/wheat_disease.h5')

In [ ]:
#define correct class order based on your training
classes = ['healthy','septoria','stripe_rust']

In [ ]:
#test image path
image_path = "/content/drive/MyDrive/wheat_leaf/Healthy/loh(1).JPG"

In [ ]:
#preprocessing image
IMG_SIZE = 256
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0
img = np.expand_dims(img, axis=0)

In [ ]:
#predict
prediction = model.predict(img)
print("Raw predictions score:", prediction)
print("Predicted class:", classes[np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Raw predictions score: [[0.05879652 0.6453125  0.295891  ]]
Predicted class: septoria
